## 복수선택형 문제

In [ ]:
from tools.core.llm_query import LLMQuery

llm = LLMQuery()
system_prompt = """당신은 프롬프트 엔지니어입니다. 아래와 같은 원리를 담아 문제를 변형하는 프롬프트를 만들려고 합니다.
### 옳은 것 → 옳지 않은 것 복수선택형

- (5지선다) 어떤 문제를 답2개/3개/4개/5개 정하고
    - 5개: 옳은 것 0개 = 본래 O선지 1개 → X선지 1개 = 옳지 않은 것 5개
    - 4개: 옳은 것 1개 = 본래 O선지 1개 그대로 사용 = 옳지 않은 것 4개
    - 3개: 옳은 것 2개 = 본래 O선지 1개 + X선지 중 1개 → O선지 = 옳지 않은 것 3개
    - 2개: 옳은 것 3개 = 본래 O선지 1개 + X선지 중 2개 → O선지 2개 = 옳지 않은 것 2개
- (4지선다) 어떤 문제를 답2개/3개/4개 할지 정하고
    - 4개: 옳은 것 0개 = 본래 O선지 1개 → X선지 1개 = 옳지 않은 것 4개
    - 3개: 옳은 것 1개 = 본래 O선지 1개 그대로 사용 = 옳지 않은 것 3개 ⇒ 단일선택형 재(변형)검증
    - 2개: 옳지 않은 것 2개 = 본래 O선지 1개 + X선지 중 1개 → O선지 = 옳지 않은 것 2개

아래 프롬프트는 반대의 상황을 위해 만들어진 프롬프트입니다.
모순 없이 위의 상황을 반영할 수 있도록 수정해주세요.
"""

target_answer_count = 3
options_ct = 5
question = {}
question['answer'] = "②"


user_prompt = f"""
당신은 25년 경력의 문제 출제 전문가입니다.

검증
- {target_answer_count}가 2 이상 {options_ct} 이하인지 확인합니다. 범위를 벗어나면 오류를 보고합니다.
- 선택지 수({options_ct})와 순서는 반드시 유지합니다.

변형 규칙
- 문제 지문은 “옳은 것을 모두 고르시오”로 바꿉니다. 그 외 본문, 수식(LaTeX), 표, 선택지 문구(변형 대상 제외)는 원형을 최대한 보존합니다.
- 목표 오답 수 = {options_ct - target_answer_count}.
  - 목표 오답 수 = 0: 원래 오답({question.get('answer', '')})을 최소 수정으로 ‘옳음’으로 뒤집습니다. 그 외 선택지는 변경하지 않습니다. 결과적으로 모든 선택지가 옳음입니다.
  - 목표 오답 수 = 1: 원래 오답을 그대로 유지합니다. 그 외 선택지는 변경하지 않습니다. 결과적으로 원래 오답 1개만 남습니다(단일선택형 재검증).
  - 목표 오답 수 ≥ 2: 원래 오답은 그대로 유지하고, 추가로 (목표 오답 수 - 1)개 만큼 원래 옳았던 선택지를 골라 최소 수정으로 ‘옳지 않음’으로 만듭니다.
- 최소 수정 원칙
  - 허용되는 변경 예: 수치/단위/부등호/최대↔최소/있다↔없다/반드시↔경우도 있다/조건의 범위·한정어 조정 등.
  - 제공된 해설의 논리 범위를 벗어나는 임의 창작 금지. 외부 사실 의존 금지.
  - 변형 대상이 아닌 선택지의 문구는 절대 수정하지 않습니다.
- 선택지 선정 가이드
  - 변형이 필요한 경우, 가장 적은 토큰 변경으로 참⇄거짓을 뒤집기 쉬운 선택지부터 우선 선택합니다.
  - 의미 일관성 유지: 변형으로 인해 다른 선택지와 모순되거나 문제 전체의 전제가 깨지지 않도록 합니다.

정답과 설명
- 정답(answer)은 변형 후 ‘옳은’ 선택지의 번호 목록입니다. 번호는 "①","②",... 형식의 문자열로, 오름차순으로 정렬합니다.
- explanation에는 모든 선택지를 순회하며 다음 형식으로 간단·명확히 기술합니다.
  - “① 옳다: 근거 …”
  - “③ 옳지 않다(원래 오답): 근거 …”
  - “⑤ 옳지 않다(변형): 변경 ‘높다→낮다’, 근거 …”
- 변형된 선택지는 반드시 어떤 단어/수치/기호를 어떻게 바꿨는지 구체적으로 한 토큰 수준으로 표기합니다.

출력 형식(JSON)
{{
  "question_id": "문제번호",
  "question": "변형된 문제(옳은 것을 모두 고르시오)",
  "options": ["① 선택지1", "② 선택지2", ...],  
  "answer": ["정답번호1", "정답번호2", ...], 
  "explanation": "① 옳다: … / ③ 옳지 않다(원래 오답): … / ⑤ 옳지 않다(변형): 변경 ‘…→…’, 근거 …"
}}

비고
- 목표 오답 수 계산식으로 일관되게 제어합니다.
  - 추가로 만들어야 할 오답 수 = {max(0, options_ct - target_answer_count - 1)}.
  - 단, {options_ct - target_answer_count} = 0이면 원래 오답을 ‘옳음’으로 뒤집고 추가 오답은 0입니다.
- 선택지 레이블(①, ②, …)과 원문 순서를 유지합니다.
- 최종적으로 옳음 개수 = {target_answer_count}, 오답 개수 = {options_ct - target_answer_count}임을 자기점검합니다.
"""
response = llm.query_openrouter(system_prompt, user_prompt)

In [ ]:
print(response)

In [ ]:
system_prompt = f"""당신은 25년 경력의 문제 출제 전문가입니다.

검증
- 목표 정답 수(= '옳지 않은 것' 개수, {target_answer_count})가 2 이상 {options_ct} 이하인지 확인합니다. 범위를 벗어나면 오류를 보고합니다.
- 선택지 수({options_ct})와 순서는 반드시 유지합니다.

변형 규칙
- 문제 지문은 “옳지 않은 것을 모두 고르시오”로 바꿉니다. 그 외 본문, 수식(LaTeX), 표, 선택지 문구(변형 대상 제외)는 원형을 최대한 보존합니다.
- 목표 오답 수 = '옳은 것' 개수 = {options_ct - target_answer_count}.
  - 목표 오답 수 = 0: 원래 오답({question.get('answer', '')})을 최소 수정으로 ‘옳지 않음’으로 뒤집습니다. 그 외 선택지는 변경하지 않습니다. 결과적으로 모든 선택지가 옳지 않음입니다.
  - 목표 오답 수 = 1: 원래 오답을 그대로 유지합니다. 그 외 선택지는 변경하지 않습니다. 결과적으로 원래 오답인 '옳은 것' 1개만 남습니다(단일선택형 재검증).
  - 목표 오답 수 ≥ 2: 원래 오답을 그대로 유지하고, 추가로 (목표 오답 수 - 1)개 만큼 원래 옳지 않았던 선택지를 골라 최소 수정으로 ‘옳음’으로 만듭니다.
- 최소 수정 원칙
  - 허용되는 변경 예: 수치/단위/부등호/최대↔최소/있다↔없다/반드시↔경우도 있다/조건의 범위·한정어 조정 등.
  - 제공된 해설의 논리 범위를 벗어나는 임의 창작 금지. 외부 사실 의존 금지.
  - 변형 대상이 아닌 선택지의 문구는 절대 수정하지 않습니다.
- 선택지 선정 가이드
  - 변형이 필요한 경우, 가장 적은 토큰 변경으로 참⇄거짓을 뒤집기 쉬운 선택지부터 우선 선택합니다.
  - 의미 일관성 유지: 변형으로 인해 다른 선택지와 모순되거나 문제 전체의 전제가 깨지지 않도록 합니다.

정답과 설명
- 정답(answer)은 변형 후 ‘옳지 않은’ 선택지의 번호 목록입니다. 번호는 "①","②",... 형식의 문자열로, 오름차순으로 정렬합니다.
- explanation에는 모든 선택지를 순회하며 다음 형식으로 간단·명확히 기술합니다.
  - “① 옳다(원래 오답): 근거 …”
  - “③ 옳지 않다: 근거 …”
  - “⑤ 옳다(변형): 변경 ‘높다→낮다’, 근거 …”
- 변형된 선택지는 반드시 어떤 단어/수치/기호를 어떻게 바꿨는지 구체적으로 한 토큰 수준으로 표기합니다.

출력 형식(JSON)
{{
  "question_id": "문제번호",
  "question": "변형된 문제(옳지 않은 것을 모두 고르시오)",
  "options": ["① 선택지1", "② 선택지2", ...],  
  "answer": ["정답번호1", "정답번호2", ...], 
  "explanation": "① 옳다(원래 오답): … / ③ 옳지 않다: … / ⑤ 옳다(변형): 변경 ‘…→…’, 근거 …"
}}

비고
- 목표 오답 수 계산식으로 일관되게 제어합니다.
  - 추가로 만들어야 할 오답 수 = {max(0, options_ct - target_answer_count - 1)}.
  - 단, {options_ct - target_answer_count} = 0이면 원래 오답을 ‘옳지 않음’으로 뒤집고 추가 오답은 0입니다.
- 선택지 레이블(①, ②, …)과 원문 순서를 유지합니다.
- 최종적으로 옳지 않음 개수 = {target_answer_count}, 오답(옳음) 개수 = {options_ct - target_answer_count}임을 자기점검합니다.
"""

### 피드백 반영

In [ ]:
atc[132]

In [ ]:
user_prompt = f"""
=========== 문제 ==========
"""
p = atc[132]
question_id = p['file_id'] + '_' + p['tag']
question = p['question']
options = p['options']
answer = p['answer']
explanation = p['explanation']

user_prompt += f"""문제번호: {question_id}
문제: {question}
선택지: {options}
답: {answer}
해설: {explanation}
"""

In [ ]:
# 입력
# - options_ct: 원문 선택지 수
# - i: 변형 후 정답(‘옳은 것’)의 개수
# - given_wrong_idx: 원래 ‘옳지 않은’ 선택지 번호(필수)

question = atc[132]
options_ct = len(options)
target_answer_count = 2
given_wrong_idx = answer

system_prompt = f"""당신은 25년 경력의 문제 출제 전문가입니다.

검증
- 목표 정답 수(= '옳지 않은 것' 개수, {target_answer_count})가 2 이상 {options_ct} 이하인지 확인합니다. 범위를 벗어나면 오류를 보고합니다.
- 선택지 수({options_ct})와 순서는 반드시 유지합니다.

변형 규칙
- 문제 지문은 “옳지 않은 것을 모두 고르시오”로 바꿉니다. 그 외 본문, 수식(LaTeX), 표, 선택지 문구(변형 대상 제외)는 원형을 최대한 보존합니다.
- 목표 오답 수 = '옳은 것' 개수 = {options_ct - target_answer_count}.
  - 목표 오답 수 = 0: 원래 오답({question.get('answer', '')})을 최소 수정으로 ‘옳지 않음’으로 뒤집습니다. 그 외 선택지는 변경하지 않습니다. 결과적으로 모든 선택지가 옳지 않음입니다.
  - 목표 오답 수 = 1: 원래 오답을 그대로 유지합니다. 그 외 선택지는 변경하지 않습니다. 결과적으로 원래 오답인 '옳은 것' 1개만 남습니다(단일선택형 재검증).
  - 목표 오답 수 ≥ 2: 원래 오답을 그대로 유지하고, 추가로 (목표 오답 수 - 1)개 만큼 원래 옳지 않았던 선택지를 골라 최소 수정으로 ‘옳음’으로 만듭니다.
- 최소 수정 원칙
  - 허용되는 변경 예: 수치/단위/부등호/최대↔최소/있다↔없다/반드시↔경우도 있다/조건의 범위·한정어 조정 등.
  - 제공된 해설의 논리 범위를 벗어나는 임의 창작 금지. 외부 사실 의존 금지.
  - 변형 대상이 아닌 선택지의 문구는 절대 수정하지 않습니다.
- 선택지 선정 가이드
  - 변형이 필요한 경우, 가장 적은 토큰 변경으로 참⇄거짓을 뒤집기 쉬운 선택지부터 우선 선택합니다.
  - 의미 일관성 유지: 변형으로 인해 다른 선택지와 모순되거나 문제 전체의 전제가 깨지지 않도록 합니다.

정답과 설명
- 정답(answer)은 변형 후 ‘옳지 않은’ 선택지의 번호 목록입니다. 번호는 "①","②",... 형식의 문자열로, 오름차순으로 정렬합니다.
- explanation에는 모든 선택지를 순회하며 다음 형식으로 간단·명확히 기술합니다.
  - “① 옳다(원래 오답): 근거 …”
  - “③ 옳지 않다: 근거 …”
  - “⑤ 옳다(변형): 변경 ‘높다→낮다’, 근거 …”
- 변형된 선택지는 반드시 어떤 단어/수치/기호를 어떻게 바꿨는지 구체적으로 한 토큰 수준으로 표기합니다.

출력 형식(JSON)
{{
  "question_id": "문제번호",
  "question": "변형된 문제(옳지 않은 것을 모두 고르시오)",
  "options": ["① 선택지1", "② 선택지2", ...],  
  "answer": ["정답번호1", "정답번호2", ...], 
  "explanation": "① 옳다(원래 오답): … / ③ 옳지 않다: … / ⑤ 옳다(변형): 변경 ‘…→…’, 근거 …"
}}

비고
- 목표 오답 수 계산식으로 일관되게 제어합니다.
  - 추가로 만들어야 할 오답 수 = {max(0, options_ct - target_answer_count - 1)}.
  - 단, {options_ct - target_answer_count} = 0이면 원래 오답을 ‘옳지 않음’으로 뒤집고 추가 오답은 0입니다.
- 선택지 레이블(①, ②, …)과 원문 순서를 유지합니다.
- 최종적으로 옳지 않음 개수 = {target_answer_count}, 오답(옳음) 개수 = {options_ct - target_answer_count}임을 자기점검합니다.
"""

In [ ]:
from tools.core.llm_query import LLMQuery

llm = LLMQuery()
response = llm.query_openrouter(system_prompt, user_prompt, model_name = 'openai/o3')
print(response)

In [ ]:
question

## 서술형 프롬프트 짜기

In [4]:
from tools.core.llm_query import LLMQuery

llm = LLMQuery()

In [5]:
system_prompt = """당신은 프롬프트 엔지니어입니다. 아래와 같은 원리를 담아 객관식 문제를 서술형 문제로 변형하는 프롬프트를 만들려고 합니다.

1. 기존 객관식 문제에서 '어떤 내용'을 서술하면 좋을지 추출 (ex. 독점적 경쟁시장에 관한 설명으로 옳지 않은 것은? -> 독점적 경쟁시장)
2. 추출한 내용을 서술형 문제로 변형 (ex. 독점적 경쟁시장에 관한 설명으로 옳지 않은 것은? -> 독점적 경쟁시장에 대해 서술하시오)
3. 1개의 옳지 않은 선지와 그 해설, 3~4개의 옳은 선지와 그 해설에서 각 선지별로 키워드 추출 (ex. ② 장기균형에서 기업들의 경제학적 이윤은 0이다. -> 장기균형, 기업의 경제학적 이윤)
4. 키워드를 포함한 서술형 문제 완성 (ex. 다음 키워드를 활용하여 독점적 경쟁시장에 관해 서술하시오.\n 키워드: 장기균형, 기업의 경제학적 이윤, ...)

단, 키워드는 10개 이하여야 합니다.
위와 같은 원리를 담아 서술형문제로 변형할 수 있는 프롬프트를 작성해주세요."""

user_prompt = ""

In [7]:
response = llm.query_openrouter(system_prompt, user_prompt, model_name = 'openai/gpt-5')
print(response)

AuthenticationError: Error code: 401 - {'error': {'message': 'User not found.', 'code': 401}}